## RQ1 - Calculo de todos os elementos buggy em ficheiros Python usando AST's

In [ ]:
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path
import ast
import astpretty
import astor
#getting full path 
#abs_path = Path('bugs.csv').absolute()
#sys.path.append(str(abs_path))
#print(str(abs_path))

In [ ]:
bugs = pd.read_csv('../../subjects/data/generated/bugs.csv', sep=',')

In [ ]:
# Converting series to String

bugs['commit_hash']= pd.Series(bugs['commit_hash'], dtype="string")
bugs['id']= pd.Series(bugs['id'], dtype="string")
bugs['type']= pd.Series(bugs['type'], dtype="string")
bugs['repo']= pd.Series(bugs['repo'], dtype="string")

## Configure cwd on git calls

In [ ]:
buggy_elements_df = pd.DataFrame()

length = bugs.shape[0]
j = 0



for i in range(length):
    sha1 = (bugs['commit_hash'].iloc[i])
    bug_id = (bugs['id'].iloc[i])
    bug_type = bugs['type'].iloc[i]
    if (sha1 == '6f66672740d1322995916dbeb1ffc2f94403baac' or sha1 == '4d260bd6545c20f3b1fefc2c843fb67d1296cb45' or sha1 == 'be70211e1b4cad4326f01f9b2f790643255a7aaa' or sha1 == 'beb7500622712186a9bf7f27fd3d8f354ac17d35' or sha1 == '19b6b634e54fc612e9b92673dd25d2c3e25a09f5' or sha1 == 'f1703607ca637c111ad362442ca2209354ff79a0' or sha1 == 'e980988e148243ed5d650317cad2870c6985f2ae' or sha1 == '05f379494b78ad5d8b6fcd0d924b3ac6aa5be92b'):
        continue
    repo_url = 'https://github.com/' + bugs['repo'].iloc[i] +'.git'
    sub_repo_url = bugs['repo'].iloc[i].split('/')
    print(sha1)
    print(repo_url)
    print(sub_repo_url[1])

    ### Clone do repositorio associado ao bug
    
    process = subprocess.Popen(['git','clone',repo_url],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste', stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output, errors = process.communicate()
    #print(r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + bugs['repo'].iloc[i])

    
    ### Fazer checkout para a versão fixed do bug - commit fixed
    
    process2 = subprocess.Popen(['git','checkout',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output, errors = process2.communicate()
  
    
    ### Obter somente os ficheiros py modificados no dado commit
    
    
    
    process3 = subprocess.Popen(['git','diff-tree','--no-commit-id','--name-only','-r',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output3, errors = process3.communicate()


    Dicionario_linhas_ficheiros = []
    for ficheiro in output3.splitlines():
        #print('NOME DO FICHEIRO')
        #print(ficheiro)
        if(ficheiro.endswith('.py')):
        
            
            ### Quais linhas foram modificadas usando o git diffn
            
            process4 = subprocess.Popen(['git','diffn',sha1 + '~1',sha1,ficheiro],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
            output4, errors = process4.communicate()
           
            
            ### Obter linhas do lado esquerdo , a vermelho, i.e eliminadas e do lado direito, a verde, i.e modificadas
            
            linhas_eliminadas = []
            linhas_modificadas = []
            for linha in output4.splitlines():
                
                if(linha[0] == ' '):
                    
                    continue
                else:
                
                    try:
                        size = len(linha.split(":",1)[0])
                        mod_string = linha.split(":",1)[0][:size - 3]
                        if(mod_string.endswith(' ')):
                            #print('Linha eliminada ' + linha)
                            reverse1 = mod_string[::-1]
                            i = 0
                            
                            for a in reverse1.strip():
                                if a.isnumeric():
                                    i = i+1
                        
                                else :
                                    break
                            linha_final = (reverse1.strip()[0:i][::-1])
                            #print('linha final ' + linha_final)
                            if not linha_final:
                                continue
                            else:
                                linhas_eliminadas.append(int(linha_final))
                            #print(linhas_eliminadas)
            
                        else:
                            #print('Linha modificada ' + linha)
                            if 'index'  in linha :
                                continue
                            else:
                                reverse2 = mod_string[::-1]
                                j = 0

                                for b in reverse2.strip():
                                    if b.isnumeric():
                                        j = j+1

                                    else :
                                        break
                                linha_final = (reverse2.strip()[0:j][::-1])
                                if not linha_final:
                                    continue
                                else:

                                    #print('linha final ' + linha_final)
                                    linhas_modificadas.append(int(linha_final))
                                    #print(linhas_modificadas)
                            
                    except IndexError:
                        print('Empty List: was an empty line')
            #print('Linhas modificadas/adicionadas ')
            #print(linhas_modificadas)
            #print('Linhas eliminadas')
            #print(linhas_eliminadas)
            
            ### Adicionar à lista o  dicionário com nome do ficheiro,linhas eliminadas e linhas modificadas
            
            Dicionario_linhas_ficheiros.append({'Ficheiro':ficheiro, 'Linhas Modificadas':linhas_modificadas, 'Linhas Eliminadas': linhas_eliminadas})
            #print(Dicionario_linhas_ficheiros)
            
            ### calcular a AST da versão anterior para as linhas eliminadas e a ast da versão atual para as linhas modificadas
                    
            
            ### fazer checkout para a versão anterior para calcular ast e usar linhas eliminadas 
            process5 = subprocess.Popen(['git','checkout',sha1 +'~1' ],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
            output5, errors = process5.communicate()
            
            ### percorrer lista de dicionários com nome do ficheiro e usar linhas eliminadas
            
            for ficheiro in  Dicionario_linhas_ficheiros:
                print(ficheiro['Ficheiro'])
                try :
                    with open(r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url[1]) + '\\' + ficheiro['Ficheiro'], "r", encoding="utf8") as source:  
                        #astpretty.pprint(ast.parse(source.read()))
                        tree = ast.parse(source.read())
                        deleted_lines = ficheiro['Linhas Eliminadas']
                        #new_line_list1 = []
                        #new_line_list3 = []
                        deleted_lines_dict = {}
                        for line in deleted_lines:
                            deleted_lines_dict[str(line)] = []



                        for node in ast.walk(tree):

                            if isinstance(node, ast.expr) or isinstance(node, ast.stmt) :

                                line = node.lineno
                                endline = node.end_lineno

                                new_line_list2 = []
                                for deleted_line in deleted_lines:   

                                    if deleted_line == line and deleted_line == endline:

                                        #print(type(node))
                                        deleted_lines_dict[str(deleted_line)].append(type(node))

                        print(deleted_lines_dict)


                        for key in deleted_lines_dict:

                            row  = {'Bug ID': bug_id,'Hash': sha1,'Bug Type': bug_type ,'Repo' :  str(sub_repo_url[1]) ,'File': ficheiro['Ficheiro'], 'Line': key, 'Values': deleted_lines_dict[key]}

                            buggy_elements_df = buggy_elements_df.append(row, ignore_index = True)
                            

                except FileNotFoundError:

                    print('File does not exist in your local repo!')
                    
                except PermissionError:
                    
                    print('No permission to open this file')
                
                except:
                
                    print('Something wrong happened')

















        

                ### fazer checkout para a versão fixed again

                #process7 = subprocess.Popen(['git','checkout',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
                #output7, errors = process7.communicate()

         


            else:
                continue
       
      

In [ ]:
 buggy_elements_df.shape

In [ ]:
buggy_elements_df.head()

In [ ]:
buggy_elements_df.to_csv('..\data\RQ1_Data.csv', index=False, encoding="utf8" )
